<a href="https://colab.research.google.com/github/stefymojica/MACC/blob/main/modelo_clinico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agente para la optimizacion de recursos en un modelo clinico
## Matematicas aplicadas y ciencias de la computacion
## Estudiante : Stefany Paola Mojica Melo

## Contextualizacion

Este agente simula una clinica privada de alta complejidad en horas de la tarde/noche, donde se concentra el mayor flujo de pacientes y la carga asistencial critica

- 1 tick corresponde 1 minutos, 20 ticks corresponde a 20 minutos y asi sucesivamente
- Hay 3 agentes:
  - pacientes
  - medicos
  - triaje
- Hay un modelo
  - Clinica donde los agentes van a interactuar
- En el triaje atienden 2 doctores y aproximadamente se demoran 10 minutos atendiendo a los pacientes
- Hay pacientes de manera aleatoria que llegan en nivel 5 y los envian nuevamente a sus casas
- Los pacientes llegan y el triaje asigna el nivel de riesgo dependiendo de la situacion del paciente
- A la clinica llegan cada 1 hora  entre 15 a 22 pacientes
- Los doctores tienen un tiempo especifico que se toman para tratar a los pacientes de acuerdo a su nivel de criticidad que le haya asignado el triaje
- los pacientes llegan y el triaje les asigana un nivel de criticidad son 5 niveles.
  - nivel 1 - situacion de riesgo vital, requiere accion inmediata
    - Minutos de atencion: 30 minutos maximo
  - nivel 2 - Emergencia. requiere atencion en un tiempo maximo de 30 minutos
    - Minutos de atencion:
  - nivel 3 - Urgencia. se debe atender en un tiempo no mayor a 30 minutos
    - Minutos de atencion:
  - nivel 4 - urgencia menor. Puede esperar hasta 120 minutos
    - Minutos de atencion:
  - nivel 5 - sin urgencia. puede esperar hasta 180 minutos
    - Minutos de atencion:

## Instalar librerias

In [1]:
%pip install mesa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 10.1 MB/s eta 0:00:00


In [2]:
import mesa
print("Versión de Mesa instalada:", mesa.__version__)

Versión de Mesa instalada: 3.3.0


In [3]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random, heapq
import statistics


## Creacion de Agentes y Modelo

In [19]:
class SchedulerManual:
    def __init__(self, model):
        self.model = model
        self.agents = []

    def add(self, agent):
        self.agents.append(agent)

    def step(self):
        random.shuffle(self.agents)
        for agent in self.agents:
            agent.step()

class Paciente(Agent):
    def __init__(self, unique_id, model):
        super().__init__(model)
        self.unique_id = unique_id
        self.prioridad = None
        self.atendido = False
        self.llegada_tick = model.tick
        self.inicio_atencion_tick = None
        self.fin_atencion_tick = None

    def step(self):
        pass

class Triaje(Agent):
    def __init__(self, unique_id, model):
        super().__init__(model)
        self.unique_id = unique_id
        self.ocupado = False
        self.paciente_actual = None
        self.tiempo_restante = 0

    def step(self):
        if self.ocupado:
            self.tiempo_restante -= 1
            if self.tiempo_restante <= 0:
                self.asignar_prioridad(self.paciente_actual)
                self.ocupado = False
                self.paciente_actual = None
            return

        # Si está libre, atiende al siguiente paciente
        if len(self.model.pacientes_sin_triaje) > 0:
            paciente = self.model.pacientes_sin_triaje.pop(0)
            self.atender(paciente)

    def atender(self, paciente):
        self.ocupado = True
        self.paciente_actual = paciente
        # Tiempo entre 5 y 10 minutos
        self.tiempo_restante = random.randint(5, 10)

    def asignar_prioridad(self, paciente):
        r = random.random()
        if   r < 0.05: paciente.prioridad = 1
        elif r < 0.15: paciente.prioridad = 2
        elif r < 0.40: paciente.prioridad = 3
        elif r < 0.20: paciente.prioridad = 4
        else:          paciente.prioridad = 5

        heapq.heappush(self.model.cola_prioridad_triaje, (paciente.prioridad, paciente.unique_id, paciente))

class Doctor(Agent):
    def __init__(self, unique_id, model):
        super().__init__(model)
        self.unique_id = unique_id
        self.ocupado = False
        self.paciente_actual = None
        self.tiempo_restante = 0

    def step(self):
        if self.ocupado:
            self.tiempo_restante -= 1
            if self.tiempo_restante <= 0:
                self.ocupado = False
                self.paciente_actual.fin_atencion_tick = self.model.tick
                self.paciente_actual = None
            return

        if len(self.model.cola_prioridad_triaje) > 0:
            _, _, paciente = heapq.heappop(self.model.cola_prioridad_triaje)
            self.atender(paciente)

    def atender(self, paciente):
        self.ocupado = True
        self.paciente_actual = paciente
        paciente.atendido = True
        paciente.inicio_atencion_tick = self.model.tick
        self.model.pacientes_atendidos += 1

        # Duración de atención según prioridad
        tiempo_por_prioridad = {1: 20, 2: 15, 3: 12, 4: 10, 5: 8}
        self.tiempo_restante = tiempo_por_prioridad[paciente.prioridad]

class Clinica(Model):
    def __init__(self, n_doctores, n_triages=2, semilla=42):
        super().__init__()
        random.seed(semilla)

        self.n_doctores = n_doctores
        self.n_triages = n_triages
        self.pacientes_sin_triaje = []
        self.cola_prioridad_triaje = []
        self.pacientes_atendidos = 0
        self.todos_pacientes = []
        self.tick = 0

        self.schedule = SchedulerManual(self)
        # se crean los doctores
        for d in range(n_doctores):
            self.schedule.add(Doctor(f"D{d+1}", self))

        # se crean los doctores del triage
        self.triages = []
        for t in range(n_triages):
            tr = Triaje(f"T{t+1}", self)
            self.triages.append(tr)
            self.schedule.add(tr)

    def _llegada_ola(self):
        if self.tick % 60 == 0:
            nuevos = random.randint(15, 22)
            for i in range(nuevos):
                pid = f"P{self.tick}_{i}"
                p = Paciente(pid, self)
                self.schedule.add(p)
                self.pacientes_sin_triaje.append(p)
                self.todos_pacientes.append(p)

    def step(self):
        self._llegada_ola()
        self.schedule.step()
        self.tick += 1

    def mostrar_estado(self):
        h = self.tick // 60
        m = self.tick % 60
        print(f"\n===== MINUTO {self.tick} ({h:02d}:{m:02d}) =====")

        # Mostrar cola de pacientes ya clasificados (esperando doctor)
        if len(self.cola_prioridad_triaje) == 0:
            print("🟢 Cola de espera para doctor: VACÍA")
        else:
            cola_legible = [(prio, pac.unique_id) for (prio, _, pac) in self.cola_prioridad_triaje]
            print("🟡 Pacientes esperando doctor (prioridad, id):", cola_legible)

        # Mostrar estado de triagistas
        for tr in self.triages:
            if tr.ocupado:
                print(f"🏥 Triaje {tr.unique_id} evaluando a {tr.paciente_actual.unique_id} "
                      f"(restan {tr.tiempo_restante} min)")
            else:
                print(f"✅ Triaje {tr.unique_id} está libre")

        # Mostrar estado de doctores
        for ag in self.schedule.agents:
            if isinstance(ag, Doctor):
                if ag.ocupado:
                    print(f"👨‍⚕️ Doctor {ag.unique_id} atendiendo a {ag.paciente_actual.unique_id} "
                          f"(restan {ag.tiempo_restante} min)")
                else:
                    print(f"✅ Doctor {ag.unique_id} está libre")

        # NUEVAS MÉTRICAS SEPARADAS
        pacientes_sin_triaje = len(self.pacientes_sin_triaje)
        pacientes_en_triaje = sum(1 for t in self.triages if t.ocupado)
        pacientes_esperando_doctor = len(self.cola_prioridad_triaje)
        total_faltantes = pacientes_sin_triaje + pacientes_en_triaje + pacientes_esperando_doctor

        print(f"📈 Total atendidos: {self.pacientes_atendidos}")
        print(f"⏳ Pacientes sin triaje: {pacientes_sin_triaje}")
        print(f"🩺 Pacientes en triaje: {pacientes_en_triaje}")
        print(f"💬 Pacientes esperando doctor: {pacientes_esperando_doctor}")
        print(f"📊 Pacientes en proceso total: {total_faltantes}")

In [20]:
if __name__ == "__main__":
    clinica = Clinica(n_doctores=2, n_triages=2,semilla=42)
    for _ in range(180):
        clinica.step()
        if clinica.tick % 30 == 0:
            clinica.mostrar_estado()


===== MINUTO 30 (00:30) =====
🟡 Pacientes esperando doctor (prioridad, id): [(2, 'P0_5'), (5, 'P0_3')]
🏥 Triaje T1 evaluando a P0_6 (restan 4 min)
🏥 Triaje T2 evaluando a P0_7 (restan 7 min)
👨‍⚕️ Doctor D2 atendiendo a P0_2 (restan 5 min)
👨‍⚕️ Doctor D1 atendiendo a P0_4 (restan 5 min)
📈 Total atendidos: 4
⏳ Pacientes sin triaje: 8
🩺 Pacientes en triaje: 2
💬 Pacientes esperando doctor: 2
📊 Pacientes en proceso total: 12

===== MINUTO 60 (01:00) =====
🟡 Pacientes esperando doctor (prioridad, id): [(2, 'P0_11'), (5, 'P0_9'), (5, 'P0_8')]
🏥 Triaje T1 evaluando a P0_13 (restan 3 min)
🏥 Triaje T2 evaluando a P0_12 (restan 3 min)
✅ Doctor D1 está libre
👨‍⚕️ Doctor D2 atendiendo a P0_7 (restan 2 min)
📈 Total atendidos: 9
⏳ Pacientes sin triaje: 2
🩺 Pacientes en triaje: 2
💬 Pacientes esperando doctor: 3
📊 Pacientes en proceso total: 7

===== MINUTO 90 (01:30) =====
🟡 Pacientes esperando doctor (prioridad, id): [(3, 'P0_15'), (3, 'P60_1'), (5, 'P0_13'), (5, 'P60_0'), (5, 'P0_9'), (5, 'P60_2')]